In [8]:
import pandas as pd

In [39]:
df_homicide = pd.read_excel('table-20.xls', engine='xlrd', skiprows=3, skipfooter=5)

df_owners = pd.read_excel('TL-354-State-Level Estimates of Household Firearm Ownership.xlsx', engine='xlrd', sheet_name='State-Level Data & Factor Score')

df_population = pd.read_excel('nst-est2019-01.xlsx', engine='xlrd', skiprows=3, skipfooter=7)

In [62]:
df_homicide.columns = [c.replace('\n', ' ') for c in df_homicide.columns]
df_homicide = df_homicide.rename({'State': 'STATE', 'Total murders1': 'Total murders'}, axis='columns')
df_homicide['STATE'] = df_homicide.STATE.map(lambda s: s.replace('3', '').replace('4', '').replace('5', ''))
df_homicide

,STATE,Total murders,Total firearms,Handguns,Rifles,Shotguns,Firearms (type unknown),Knives or cutting instruments,Other weapons,"Hands, fists, feet, etc.2"
0,Alabama,4,3,3,0,0,0,0,1,0
1,Alaska,69,44,17,1,6,20,8,5,12
2,Arizona,337,213,170,6,6,31,47,67,10
3,Arkansas,231,177,87,10,5,75,21,29,4
4,California,1679,1142,762,34,26,320,252,183,102
5,Colorado,209,135,83,5,3,44,32,30,12
6,Connecticut,104,65,11,1,0,53,15,17,7
7,Delaware,48,40,13,0,0,27,2,3,3
8,District of Columbia,166,136,52,0,0,84,18,2,10
9,Florida,1,0,0,0,0,0,0,1,0


In [55]:
df_owners = df_owners[df_owners.Year.isin(['2016'])].reset_index(drop=True)
df_owners

,FIP,Year,STATE,HFR,HFR_se,universl,permit,Fem_FS_S,Male_FS_S,BRFSS,GALLUP,GSS,PEW,HuntLic,GunsAmmo,BackChk,PewQChng,BS1,BS2,BS3
0,1,2016,Alabama,0.528,0.028,0,0,0.646341,0.711538,-9.0,-9.0,0.471448,0.649361,0.341758,-0.219699,0.270898,1,12.0,12.0,12.0
1,2,2016,Alaska,0.572,0.032,0,0,0.369565,0.653061,-9.0,-9.0,-9.000000,0.666125,0.657791,0.740572,1.937272,1,12.0,12.0,12.0
2,4,2016,Arizona,0.360,0.028,0,0,0.409091,0.647716,-9.0,-9.0,0.372834,0.387098,0.243218,0.071988,-0.447685,1,12.0,12.0,12.0
3,5,2016,Arkansas,0.518,0.028,0,0,0.510638,0.625604,-9.0,-9.0,0.480983,0.661608,0.363895,0.093631,0.308392,1,12.0,12.0,12.0
4,6,2016,California,0.163,0.028,1,1,0.182548,0.428571,-9.0,-9.0,0.186860,0.240008,0.160671,-1.473130,-0.898778,1,12.0,12.0,12.0
5,8,2016,Colorado,0.379,0.028,1,0,0.337165,0.578831,-9.0,-9.0,0.322770,0.476516,0.281931,0.208845,0.814156,1,12.0,12.0,12.0
6,9,2016,Connecticut,0.188,0.028,1,1,0.078261,0.358156,-9.0,-9.0,0.245675,0.272317,0.188097,-0.843628,-0.439763,1,12.0,12.0,12.0
7,10,2016,Delaware,0.387,0.028,1,0,0.428571,0.571429,-9.0,-9.0,0.381304,0.499979,0.231977,-0.676874,-0.209439,1,12.0,12.0,12.0
8,12,2016,Florida,0.288,0.028,0,0,0.353690,0.591430,-9.0,-9.0,0.232072,0.374913,0.127689,-0.777549,-0.222022,1,12.0,12.0,12.0
9,13,2016,Georgia,0.377,0.028,0,0,0.472669,0.659381,-9.0,-9.0,0.310040,0.472258,0.354964,-0.916168,-0.801952,1,12.0,12.0,12.0


In [56]:
df_population = df_population.rename({'Unnamed: 0': 'STATE'}, axis='columns')
df_population['STATE'] = df_population.STATE.map(lambda r: r.replace('.', ''))
df_population

,STATE,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,United States,308745538,308758105,309321666,311556874,313830990,315993715,318301008,320635163,322941311,324985539,326687501,328239523
1,Northeast,55317240,55318443,55380134,55604223,55775216,55901806,56006011,56034684,56042330,56059240,56046620,55982803
2,Midwest,66927001,66929725,66974416,67157800,67336743,67560379,67745167,67860583,67987540,68126781,68236628,68329004
3,South,114555744,114563030,114866680,116006522,117241208,118364400,119624037,120997341,122351760,123542189,124569433,125580448
4,West,71945553,71946907,72100436,72788329,73477823,74167130,74925793,75742555,76559681,77257329,77834820,78347268
5,Alabama,4779736,4780125,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185
6,Alaska,710231,710249,713910,722128,730443,737068,736283,737498,741456,739700,735139,731545
7,Arizona,6392017,6392288,6407172,6472643,6554978,6632764,6730413,6829676,6941072,7044008,7158024,7278717
8,Arkansas,2915918,2916031,2921964,2940667,2952164,2959400,2967392,2978048,2989918,3001345,3009733,3017804
9,California,37253956,37254519,37319502,37638369,37948800,38260787,38596972,38918045,39167117,39358497,39461588,39512223


In [63]:
df = df_homicide.merge(df_owners, on='STATE').merge(df_population, on='STATE')
df

,STATE,Total murders,Total firearms,Handguns,Rifles,Shotguns,Firearms (type unknown),Knives or cutting instruments,Other weapons,"Hands, fists, feet, etc.2",...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Alabama,4,3,3,0,0,0,0,1,0,...,4785437,4799069,4815588,4830081,4841799,4852347,4863525,4874486,4887681,4903185
1,Alaska,69,44,17,1,6,20,8,5,12,...,713910,722128,730443,737068,736283,737498,741456,739700,735139,731545
2,Arizona,337,213,170,6,6,31,47,67,10,...,6407172,6472643,6554978,6632764,6730413,6829676,6941072,7044008,7158024,7278717
3,Arkansas,231,177,87,10,5,75,21,29,4,...,2921964,2940667,2952164,2959400,2967392,2978048,2989918,3001345,3009733,3017804
4,California,1679,1142,762,34,26,320,252,183,102,...,37319502,37638369,37948800,38260787,38596972,38918045,39167117,39358497,39461588,39512223
5,Colorado,209,135,83,5,3,44,32,30,12,...,5047349,5121108,5192647,5269035,5350101,5450623,5539215,5611885,5691287,5758736
6,Connecticut,104,65,11,1,0,53,15,17,7,...,3579114,3588283,3594547,3594841,3594524,3587122,3578141,3573297,3571520,3565287
7,Delaware,48,40,13,0,0,27,2,3,3,...,899593,907381,915179,923576,932487,941252,948921,956823,965479,973764
8,Florida,1,0,0,0,0,0,0,1,0,...,18845537,19053237,19297822,19545621,19845911,20209042,20613477,20963613,21244317,21477737
9,Georgia,445,367,305,14,3,45,34,35,9,...,9711881,9802431,9901430,9972479,10067278,10178447,10301890,10410330,10511131,10617423


In [64]:
df.columns

Index([                        'STATE',                 'Total murders',
                      'Total firearms',                      'Handguns',
                              'Rifles',                      'Shotguns',
             'Firearms (type unknown)', 'Knives or cutting instruments',
                       'Other weapons',     'Hands, fists, feet, etc.2',
                                 'FIP',                          'Year',
                                 'HFR',                        'HFR_se',
                            'universl',                        'permit',
                            'Fem_FS_S',                     'Male_FS_S',
                               'BRFSS',                        'GALLUP',
                                 'GSS',                           'PEW',
                             'HuntLic',                      'GunsAmmo',
                             'BackChk',                      'PewQChng',
                                 'BS1',            

In [69]:
df['Murder rate per 100k'] = df['Total murders'] * 100000 / df[2019]
df['Gun murder rate per 100k'] = df['Total firearms'] * 100000 / df[2019]

In [70]:
df[['STATE', 2019, 'Total murders', 'Total firearms', 'HFR', 'Murder rate per 100k', 'Gun murder rate per 100k']]

,STATE,2019,Total murders,Total firearms,HFR,Murder rate per 100k,Gun murder rate per 100k
0,Alabama,4903185,4,3,0.528,0.081580,0.061185
1,Alaska,731545,69,44,0.572,9.432092,6.014668
2,Arizona,7278717,337,213,0.360,4.629937,2.926340
3,Arkansas,3017804,231,177,0.518,7.654573,5.865192
4,California,39512223,1679,1142,0.163,4.249318,2.890245
5,Colorado,5758736,209,135,0.379,3.629269,2.344264
6,Connecticut,3565287,104,65,0.188,2.917016,1.823135
7,Delaware,973764,48,40,0.387,4.929326,4.107771
8,Florida,21477737,1,0,0.288,0.004656,0.000000
9,Georgia,10617423,445,367,0.377,4.191224,3.456583
